<a href="https://colab.research.google.com/github/UsamaGM/AI-ML/blob/main/generative_ai_rnn_model_romeo_and_juliet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lets make a Generative Model today

### Start with importing the necessary librarires

In [ ]:
import os
import keras
import numpy as np
import tensorflow as tf
from keras.preprocessing import sequence

### Next, we need to download the script that we are going to train our model on.
- In this example, we are going to use Romeo and Juliet by William Shakespeare.

### Lets start by downloading the script file from google drive

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 1s 0us/step


### We can also use any script file from our local storage by using the following method
- For this example, we are not going to use this particular method

In [ ]:
# from google.colab import files
# path_to_file = list(files.upload().keys())[0]

### Lets load the script
- I am saving the script in a variable
- Also, lets see how many characters does our script contain

In [ ]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


### Lets print a random text snippet from the script
- Believe me this was not random at all

In [ ]:
print(text[479:998])

We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.


# Preprocessing the data

### Now that we have the script, we need to convert it into a form that we can feed into our model
- First, we are calculating the total number of unique **characters** present in our script and saving it into *vocab* variable
- Then, I have defined a method to convert the text in our script into integer array
- We will use the method to convert the input text into ints and feed to our model

In [ ]:
vocab = sorted(set(text))

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

### Lets use this method to convert our script into integer representation

In [ ]:
text_as_int = text_to_int(text)

### Lets see an example of how our text is converted into integer arrays

In [ ]:
print('Text is: ', text[:13])
print('Encoded as: ', text_to_int(text[:13]))

Text is:  First Citizen
Encoded as:  [18 47 56 57 58  1 15 47 58 47 64 43 52]


### Now, lets define another method to convert integer back into text
- This method will be used to convert the output from our model back into text form

In [ ]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

# Creating and compiling our model

### Lets define the parameters for our model
- *seq_length* is the number of characters that a single text sequence will contain when being fed into the model
- *examples_per_epoch* is the number of examples/text snippets that will be created after dividing the total text into smaller chunks.
  - Notice, that each chunk of the text will have a length of 101 characters as described by *seq_length*
  - We will feed 100 characters into the model and it will give us 100 characters in return
  - These 100 characters will be 99 characters from our input + 1 that the model predicts should come next
  - Example: Input: `Hell` Output: `ello`
- Lets finally divide our script into smaller chunks using tensorflow's built in method

In [ ]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

### Next, we will create batches of data
- Notice that we are creating batches of 101 characters, this is due to the same reason as described in the previous section
- Also notice that we are setting `drop_remainder=True`. This is because we want to trim the text to 101 characters and any remaining characters will not be used

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

### Next, we will split the 101 characters into two parts:
- The first hundred characters for input text
- The last hundred characters for target text (expected output text)

In [ ]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

### Lets take a look at 2 examples of input and target text

In [ ]:
for x, y in dataset.take(2):
  print('\n\nExample\n')
  print('INPUT')
  print(int_to_text(x))
  print('\nTARGET')
  print(int_to_text(y))



Example

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

TARGET
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


Example

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

TARGET
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


### Lets finally set the paramters for our model
- `BATCH_SIZE` is the number of snippets that will be feed into the model in a single go
- `VOCAB_SIZE` is the number of unique characters in our script
- `EMBEDDING_DIM` is the number of dimensions that a single character is going to have
- `RNN_UNITS` is the
- `BUFFER_SIZE` is the number of characters that will be held into buffer at any instance

### Lets use these parameters to shuffle our data and create batches

In [ ]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 256
RNN_UNITS = 1024
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

### Building the model
- I will use a method to build a model for the given parameters
- This is handy as we will create a new model once have trained our current model

### I am using the parameters defined above to create the model
- Lets take a look at the summary of the model

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
      tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
      tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5330241 (20.33 MB)
Trainable params: 5330241 (20.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Lets use our model without training to make predictions
- This will help us understand the output format of our model

In [ ]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape)

(64, 100, 65)


Lets print the output

In [ ]:
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 3.77869210e-03 -3.25156888e-03 -1.34758756e-03 ...  1.98414247e-03
    5.96947502e-03  5.15735289e-03]
  [ 5.47010638e-03 -2.56635691e-03 -1.00021728e-03 ... -9.69405519e-04
    9.45107639e-03  3.27426149e-03]
  [ 4.80640866e-03  5.12239151e-03 -7.33136851e-03 ...  1.90335559e-04
    5.45757217e-03  4.54434566e-03]
  ...
  [ 8.42446648e-03 -8.75267480e-03  2.89455289e-04 ... -1.85202854e-03
    8.89510103e-03  1.10856909e-02]
  [ 5.38608246e-03 -1.82073796e-03 -5.50154597e-03 ... -1.68352120e-03
    9.06787161e-03  7.35846069e-03]
  [ 5.45546552e-03 -1.24247489e-03 -6.97036623e-04 ... -1.15101994e-03
    1.85231841e-03  7.19488971e-03]]

 [[ 1.09037256e-03  9.92627232e-04  1.02701329e-03 ... -2.40163086e-03
    6.67476910e-04  3.86962714e-03]
  [-5.65607939e-03 -2.72559933e-04  2.09835451e-03 ...  1.98012916e-04
    6.29216316e-04  6.29244652e-03]
  [-2.48823944e-03  6.81281788e-04  2.68362369e-03 ... -2.36968184e-03
    7.44560501e-04  9.22726281e-03]
  ...
  [ 5.787

Lets print the first element of the 3 dimensional array and see how it looks like
- It contains 100 tensors, the length of sequence, each of size 65, the total number of characters in our vocabulary

In [ ]:
pred = example_batch_predictions[0]
print(len(pred))
print(pred)

100
tf.Tensor(
[[ 0.00377869 -0.00325157 -0.00134759 ...  0.00198414  0.00596948
   0.00515735]
 [ 0.00547011 -0.00256636 -0.00100022 ... -0.00096941  0.00945108
   0.00327426]
 [ 0.00480641  0.00512239 -0.00733137 ...  0.00019034  0.00545757
   0.00454435]
 ...
 [ 0.00842447 -0.00875267  0.00028946 ... -0.00185203  0.0088951
   0.01108569]
 [ 0.00538608 -0.00182074 -0.00550155 ... -0.00168352  0.00906787
   0.00735846]
 [ 0.00545547 -0.00124247 -0.00069704 ... -0.00115102  0.00185232
   0.00719489]], shape=(100, 65), dtype=float32)


Lets print the first sequence of the output (The first element of the array)

In [ ]:
time_pred = pred[0]
print(len(time_pred))
print(time_pred)

65
tf.Tensor(
[ 3.7786921e-03 -3.2515689e-03 -1.3475876e-03 -6.4116102e-03
 -1.3390678e-03 -2.0001805e-03 -1.8254002e-03 -1.1850580e-03
  2.8794236e-04  2.6290691e-03 -4.2538685e-03 -1.2449514e-03
 -2.0209318e-03 -2.7280648e-03  7.6773660e-03  5.3662136e-03
 -1.6212114e-05  2.6481235e-03  3.0350443e-03  1.3088896e-03
  2.8666728e-03 -4.4045663e-03 -3.2891580e-03  2.3083936e-03
  3.1935032e-03  2.0685693e-04  2.2611208e-04 -3.3180050e-03
 -4.5483224e-03 -3.8255309e-03 -7.8752637e-06 -6.1637913e-03
  4.6982465e-04 -7.9920818e-04  8.6926785e-04  1.7111460e-03
  4.0777801e-03 -1.7489472e-03 -3.0875271e-03 -2.5282311e-03
 -1.4765607e-04  6.6792029e-03  5.5941348e-03  1.7698761e-03
 -4.9879057e-03 -8.8077364e-04  2.3747329e-04  1.3650558e-03
  3.7870496e-03  2.0191488e-03  8.8639080e-04  3.2697653e-03
 -2.1529431e-04 -2.5701874e-03  3.6191905e-04 -2.9668782e-04
 -1.5065344e-03  3.0348403e-04  5.8779465e-03  8.4602442e-03
  7.5698420e-03 -8.5325567e-03  1.9841425e-03  5.9694750e-03
  5.157352

### Now, lets take samples of data
- This is just to see all the characters that our model has predicted **without training**

In [ ]:
sampled_indices = tf.random.categorical(pred, num_samples=1)
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]

predicted_chars = int_to_text(sampled_indices)
print(predicted_chars)

-tdzWR.QGdZUN!eKcLdN:ZjEiE kVBRvJdg.vF:'Tu
sBWRHplO!npi
k,?vtHZWmvYWwnDLiWcylEvPD.3TtZfZD$3DI&BJpe
?


### Define the loss function
- As tensorflow does not have a loss function to deal with 3 dimensional outputs
- I have created a simple loss function using tensorflow's `sparse_categorical_crossentropy` loss function

In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

### Next, lets compile our model
- I have used `adam` as optimizer function and the loss function defined earlier

In [ ]:
model.compile(optimizer='adam', loss=loss)

### Creating checkpoints
- This is essential as once we have trained our model, we will create a new model and we will need the new model to have the parameters of the old one
- This is like writing down notes for future reference, as we humans do it
- You will se the use of this later

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True
)

### Now is the time to train our model
- I have used 100 epochs as overfitting is not going to be the issue here
- Notice that I have used `checkpoint_callback` function to save the checkpoints while training

In [ ]:
history = model.fit(data, epochs=100, callbacks=[checkpoint_callback])

Epoch 1/100
172/172 [==============================] - 15s 65ms/step - loss: 2.5716
Epoch 2/100
172/172 [==============================] - 12s 62ms/step - loss: 1.8590
Epoch 3/100
172/172 [==============================] - 12s 62ms/step - loss: 1.6132
Epoch 4/100
172/172 [==============================] - 12s 63ms/step - loss: 1.4864
Epoch 5/100
172/172 [==============================] - 13s 64ms/step - loss: 1.4105
Epoch 6/100
172/172 [==============================] - 13s 65ms/step - loss: 1.3551
Epoch 7/100
172/172 [==============================] - 13s 65ms/step - loss: 1.3115
Epoch 8/100
172/172 [==============================] - 13s 66ms/step - loss: 1.2730
Epoch 9/100
172/172 [==============================] - 13s 66ms/step - loss: 1.2368
Epoch 10/100
172/172 [==============================] - 13s 67ms/step - loss: 1.2007
Epoch 11/100
172/172 [==============================] - 13s 67ms/step - loss: 1.1654
Epoch 12/100
172/172 [==============================] - 13s 67ms/step - lo

### Now, lets create a new model with a batch size of 1
- This is the model we will be using to make predictions
- We are setting batch size as 1 as we will be giving only 1 input text sequence

In [ ]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

### Load the weights from the previous model
- This is the reason of creating checkpoints in the previous part
- All the learnings of our model are put into our new model
- Notice that I am building the model with tensorshape of 1, None
  - This is because we will be providing only 1 text input to the model and it could be of any size

In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

### Generate text
- I have created a method that receives text, preprocesses it, makes transformations, feeds into the model and return the output of specified length

In [ ]:
def generate_text(model, start_string):
  num_generate = 1000

  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  temperature = 0.1

  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)

    predictions = predictions / temperature

    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

### Finally, it is time to generate some text

In [ ]:
inp = input("Type the starting string: ")
print(generate_text(model, inp))

Type the starting string: juliet
juliet, feed from my soul.
Cousin of Buckingham, and you say you, sir,
I do repent me; read not a word with you.

LUCIO:
A little falter, she shall be shriek
when you have said 'sh, now 'tis away:
If ever sorrower'd like subjects,
Yet that, then thus I could formake me faint.
Cursue that I should come to me again.

YORK:
I shuns your father's scept put off.
Is't all the world goes have done the thing
Unhe creature thee.

GRENIO:
Good more or some other house of poor selvier.

Second Senator:
Come, come, he's dead!

QUEEN ELIZABETH:
I never did her and so great a ceremonth as they can go too,
Una virginallary, speed up:
Ne'er shall make one certainly thou art executed,
and the table, now in peace than none.

GREMIO:
Take he, promise them snot.

PAULINA:
Not so good, my lord.

DUKE VINCENTIO:
There's some infaction of the deep.

LUCENTIO:
Here, madam:
'Hic ibot, though I the sight and ight
acquaint him, where he would not do that ever
He heard the name of

In [ ]:
inp2 = input("Type your starting string: ")
print(generate_text(model, inp2))

Type your starting string: juliet
juliet,
And thou shalt tell the process of their death.
Meantime to make his state and part,
And I will shake thy nose and my good shift's depart.

DUKE VINCENTIO:
Not as thou art, not what's the matter.
Provost, how came battled with
a sudden cloak! and all this is len to death, her with a fair demand?

Provost:
Now, my lord, give ules here.

GONZALO:
No, sir, I say his horse comes, with humily and of preceprehall fly to her blood. Lest thou in about?
For what are you, sir? Has he done the time,
I with immoditation of bliss and polt men come to her.

PETRUCHIO:
I swear I'll cuff you, if you hault
to the wall: the immedial posence
Which now's upon't.

AUFIDIUS:
Bling me as to speak with him.

Pedant:
Ay, sir, the fool.
We have been supple and living fear: hang from his father?

Pedant:
Ay, sir, the fool. Was not a mind to hear? I am your either heart.
Pardon it, then say so much honour.

MENENIUS:
There was a time when all the body's members
Rebell'd a

In [ ]:
inp3 = input("Type your starting string: ")
print(generate_text(model, inp3))

Type your starting string: juliet
juliet, and thy state may do appear,
Their needless vouches? Custom calls me to't:
What custom wills, in all things should be proud,
Here shall not stay to choler them by the worst.

GEORGE:
What time is more, or not to be found.

ROMEO:
Here serve to tell your person and my life,
And I will make the work about my new give unto you to make me make her stand
The first departingham and all the world.

GLOUCESTER:
The gates made proceed on one another.

CURTIS:
And were I spill this?

BUCKINGHAM:
Good to know how shall we hear their witness:
If it be so, then to dispatch my hands:
I will not farther better with her cheek.

BENVOLIO:
A cold and other part of bats:
If every fair prophecy live
To his contentious land.
O, which is this true so?

MONTAGUE:
Then we will prove the forfeits on the Tower,
An be as present for a place,
Have done the time with purpose and to pierce and one half so gentle by secret honour to the ground,
And make him all afactors from